In [12]:
import sys
import os

PROJECT_ROOT = os.path.abspath("..")   # go from TRAINING → project root
sys.path.insert(0, PROJECT_ROOT)

print("Project root added:", PROJECT_ROOT)


Project root added: d:\imp\certificate\project\Working-Project\EMPLOYEE\Time-Project-Analysis


In [13]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from models.attendance_models.lr_model import AttendanceLinearRegression
from models.attendance_models.lstm_model import AttendanceLSTM
from models.attendance_models.prophet_model import AttendanceProphet




Importing plotly failed. Interactive plots will not work.


In [14]:
# Load attendance data
df = pd.read_csv("../data/attendance/attendance_generated.csv")
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values("date")

# Convert attendance to probability if needed
# present=1, half-day=0.5, absent=0
def convert(att):
    if att == 1:
        return 1.0
    elif att == 2:
        return 0.5
    return 0.0

df["attendance"] = df["attendance"].apply(convert)


In [15]:
train_size = int(len(df) * 0.8)
train_df = df.iloc[:train_size]
test_df  = df.iloc[train_size:]


In [18]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

def regression_metrics(y_true, y_pred):
    y_true = np.array(y_true, dtype=float)
    y_pred = np.array(y_pred, dtype=float)

    return {
        "MAE": mean_absolute_error(y_true, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_true, y_pred)),
        "MAPE": np.mean(np.abs((y_true - y_pred) / (y_true + 1e-6))) * 100
    }


In [23]:
from models.attendance_models.lr_model import AttendanceLinearRegression
import pandas as pd
import numpy as np

lr_model = AttendanceLinearRegression()
lr_model.train(train_df)

lr_preds = []
lr_actual = []

MIN_HISTORY = 35  # must be > lag_30

for i in range(MIN_HISTORY, len(test_df)):
    temp_df = pd.concat([train_df, test_df.iloc[:i]])
    pred = lr_model.predict(temp_df)
    lr_preds.append(pred)
    lr_actual.append(test_df.iloc[i]["attendance"])

lr_results = regression_metrics(lr_actual, lr_preds)
lr_results


Linear Regression trained - MAE: 0.1259


{'MAE': 0.16603175939973372,
 'RMSE': 0.3386034419656406,
 'MAPE': 4681095.518920658}

In [20]:
import numpy as np
import pandas as pd
from models.attendance_models.lstm_model import AttendanceLSTM

# Initialize & train
lstm_model = AttendanceLSTM()
lstm_model.train(train_df)

lstm_preds = []
lstm_actual = []

SEQUENCE_LENGTH = 30

for i in range(SEQUENCE_LENGTH, len(test_df)):
    temp_df = pd.concat([train_df, test_df.iloc[:i]])
    pred = lstm_model.predict(temp_df)
    
    lstm_preds.append(pred)
    lstm_actual.append(test_df.iloc[i]["attendance"])

# Metrics
lstm_results = regression_metrics(lstm_actual, lstm_preds)
lstm_results


C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


LSTM model trained - Final loss: 0.0689


{'MAE': 0.1531834791346294,
 'RMSE': 0.2981134984194244,
 'MAPE': 9083002.062950935}

In [21]:
import numpy as np
from models.attendance_models.prophet_model import AttendanceProphet

prophet_model = AttendanceProphet()
prophet_model.train(train_df)

# Prophet predicts future in one shot
prophet_preds, _ = prophet_model.predict(train_df, days=len(test_df))

# Align lengths safely
prophet_actual = test_df["attendance"].values[:len(prophet_preds)]

prophet_results = regression_metrics(prophet_actual, prophet_preds)
prophet_results


20:36:57 - cmdstanpy - INFO - Chain [1] start processing
20:36:57 - cmdstanpy - INFO - Chain [1] done processing


Prophet model trained successfully


{'MAE': 0.11179326815553771,
 'RMSE': 0.25204952978298995,
 'MAPE': 6132941.898451511}

In [24]:
comparison = pd.DataFrame([
    {"Model": "Linear Regression", **lr_results},
    {"Model": "LSTM", **lstm_results},
    {"Model": "Prophet", **prophet_results}
])

comparison


,Model,MAE,RMSE,MAPE
0,Linear Regression,0.166032,0.338603,4.681096e+06
1,LSTM,0.153183,0.298113,9.083002e+06
2,Prophet,0.111793,0.252050,6.132942e+06
